In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

matplotlib.rc("font", family = "Malgun Gothic")
matplotlib.rc("axes", unicode_minus = False)
pd.set_option('display.max_columns', 500)

In [2]:
df_raw = pd.read_csv("data/users_itemsCategory_POS_v2.csv")
df_raw.head()

,idUser,idPOS,SalesDate,Price,간편식사_도시락,간편식사_샌드위치,간편식사_주먹밥,과자류_껌/초,과자류_빵/디,과자류_스낵,식품_가공식,식품_식재료,식품_안주류,아이스크_아이스,음료_아이스,음료_유제품,음료_음료,즉석조리_베이커,즉석조리_즉석커,즉석조리_튀김류,SalesYear,SalesMonth,SalesDay,SalesWeekday,SalesTime,Gender,AgeGroup
0,U100001,U100001-Y2021M1D10-P10008,2021-01-10,1000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2021,1,10,6,06:16:24,남성,40대
1,U100001,U100001-Y2021M1D3-P10004,2021-01-03,6500,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2021,1,3,6,19:08:12,남성,40대
2,U100001,U100001-Y2021M1D7-P10006,2021-01-07,750,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2021,1,7,3,00:42:18,남성,40대
3,U100277,U100277-Y2022M10D11-P10324,2022-10-11,5300,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,2022,10,11,1,00:09:52,여성,30대
4,U100277,U100277-Y2022M10D11-P10325,2022-10-11,1500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2022,10,11,1,13:45:07,여성,30대


In [3]:
df_raw = pd.get_dummies(df_raw, columns = ["Gender", "AgeGroup"])
df_raw.head()

,idUser,idPOS,SalesDate,Price,간편식사_도시락,간편식사_샌드위치,간편식사_주먹밥,과자류_껌/초,과자류_빵/디,과자류_스낵,식품_가공식,식품_식재료,식품_안주류,아이스크_아이스,음료_아이스,음료_유제품,음료_음료,즉석조리_베이커,즉석조리_즉석커,즉석조리_튀김류,SalesYear,SalesMonth,SalesDay,SalesWeekday,SalesTime,Gender_남성,Gender_여성,AgeGroup_10대,AgeGroup_20대,AgeGroup_30대,AgeGroup_40대,AgeGroup_50대,AgeGroup_60대이상
0,U100001,U100001-Y2021M1D10-P10008,2021-01-10,1000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2021,1,10,6,06:16:24,1,0,0,0,0,1,0,0
1,U100001,U100001-Y2021M1D3-P10004,2021-01-03,6500,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2021,1,3,6,19:08:12,1,0,0,0,0,1,0,0
2,U100001,U100001-Y2021M1D7-P10006,2021-01-07,750,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2021,1,7,3,00:42:18,1,0,0,0,0,1,0,0
3,U100277,U100277-Y2022M10D11-P10324,2022-10-11,5300,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,2022,10,11,1,00:09:52,0,1,0,0,1,0,0,0
4,U100277,U100277-Y2022M10D11-P10325,2022-10-11,1500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2022,10,11,1,13:45:07,0,1,0,0,1,0,0,0


In [4]:
df_weather_raw = pd.read_csv("data/weather_v3.csv")
df_weather_raw.head()

,dateObs,dateObs_y,dateObs_m,dateObs_d,TempAvg,TempMin,hhmiTempMin,TempMax,hhmiTempMax,hhmiTempMin_h,hhmiTempMin_m,hhmiTempMax_h,hhmiTempMax_m,classRainfall,hourRainfall,RainfallAmount,discomfortIndex,HumidAvg,SnowloadMax,hhmiSnowloadMax,hhmiSnowloadMax_h,hhmiSnowloadMax_m
0,2020-01-01,2020,1,1,-2.2,-6.5,00:01:00,0.3,14:57:00,0.0,1.0,14,57,약한비,9.83,0.1,전원쾌적,64.4,0.0,0.0,0.0,0.0
1,2020-01-02,2020,1,2,1.0,-0.7,00:01:00,3.8,15:00:00,0.0,1.0,15,0,무강수,0.00,0.0,전원쾌적,65.4,0.0,0.0,0.0,0.0
2,2020-01-03,2020,1,3,-0.1,-3.4,08:07:00,4.6,15:47:00,8.0,7.0,15,47,무강수,0.00,0.0,전원쾌적,56.9,0.0,0.0,0.0,0.0
3,2020-01-04,2020,1,4,1.2,-2.8,05:12:00,6.1,14:50:00,5.0,12.0,14,50,무강수,0.00,0.0,전원쾌적,50.8,0.0,0.0,0.0,0.0
4,2020-01-05,2020,1,5,1.3,-3.2,08:21:00,6.6,14:53:00,8.0,21.0,14,53,무강수,0.00,0.0,전원쾌적,45.6,0.0,0.0,0.0,0.0


In [5]:
df_raw.shape

(215618, 33)

In [6]:
df_weather_raw.shape

(1216, 22)

In [7]:
df_raw.SalesDate = pd.to_datetime(df_raw.SalesDate, format = "%Y-%m-%d")
df_weather_raw.dateObs = pd.to_datetime(df_weather_raw.dateObs, format = "%Y-%m-%d")

In [8]:
df_raw

,idUser,idPOS,SalesDate,Price,간편식사_도시락,간편식사_샌드위치,간편식사_주먹밥,과자류_껌/초,과자류_빵/디,과자류_스낵,식품_가공식,식품_식재료,식품_안주류,아이스크_아이스,음료_아이스,음료_유제품,음료_음료,즉석조리_베이커,즉석조리_즉석커,즉석조리_튀김류,SalesYear,SalesMonth,SalesDay,SalesWeekday,SalesTime,Gender_남성,Gender_여성,AgeGroup_10대,AgeGroup_20대,AgeGroup_30대,AgeGroup_40대,AgeGroup_50대,AgeGroup_60대이상
0,U100001,U100001-Y2021M1D10-P10008,2021-01-10,1000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2021,1,10,6,06:16:24,1,0,0,0,0,1,0,0
1,U100001,U100001-Y2021M1D3-P10004,2021-01-03,6500,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,2021,1,3,6,19:08:12,1,0,0,0,0,1,0,0
2,U100001,U100001-Y2021M1D7-P10006,2021-01-07,750,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2021,1,7,3,00:42:18,1,0,0,0,0,1,0,0
3,U100277,U100277-Y2022M10D11-P10324,2022-10-11,5300,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,2022,10,11,1,00:09:52,0,1,0,0,1,0,0,0
4,U100277,U100277-Y2022M10D11-P10325,2022-10-11,1500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,2022,10,11,1,13:45:07,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215613,U101098,U101098-Y2022M9D19-P10151,2022-09-19,5200,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,2022,9,19,0,19:03:10,1,0,0,0,1,0,0,0
215614,U101098,U101098-Y2022M9D23-P10154,2022-09-23,3700,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,2022,9,23,4,14:13:40,1,0,0,0,1,0,0,0
215615,U101098,U101098-Y2022M9D29-P10157,2022-09-29,2900,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,2022,9,29,3,09:24:10,1,0,0,0,1,0,0,0
215616,U101098,U101098-Y2022M9D30-P10158,2022-09-30,10300,0,0,0,0,0,0,0,2,0,1,0,0,0,0,0,0,2022,9,30,4,15:47:40,1,0,0,0,1,0,0,0


In [9]:
df_raw.columns

Index(['idUser', 'idPOS', 'SalesDate', 'Price', '간편식사_도시락', '간편식사_샌드위치',
       '간편식사_주먹밥', '과자류_껌/초', '과자류_빵/디', '과자류_스낵', '식품_가공식', '식품_식재료',
       '식품_안주류', '아이스크_아이스', '음료_아이스', '음료_유제품', '음료_음료', '즉석조리_베이커',
       '즉석조리_즉석커', '즉석조리_튀김류', 'SalesYear', 'SalesMonth', 'SalesDay',
       'SalesWeekday', 'SalesTime', 'Gender_남성', 'Gender_여성', 'AgeGroup_10대',
       'AgeGroup_20대', 'AgeGroup_30대', 'AgeGroup_40대', 'AgeGroup_50대',
       'AgeGroup_60대이상'],
      dtype='object')

In [10]:
df_raw = df_raw[['idUser', 'SalesYear', 'SalesMonth', 'Price', '간편식사_도시락', '간편식사_샌드위치',
       '간편식사_주먹밥', '과자류_껌/초', '과자류_빵/디', '과자류_스낵', '식품_가공식', '식품_식재료',
       '식품_안주류', '아이스크_아이스', '음료_아이스', '음료_유제품', '음료_음료', '즉석조리_베이커',
       '즉석조리_즉석커', '즉석조리_튀김류', 'Gender_남성', 'Gender_여성', 'AgeGroup_10대',
       'AgeGroup_20대', 'AgeGroup_30대', 'AgeGroup_40대', 'AgeGroup_50대',
       'AgeGroup_60대이상']]
df_raw.head()

,idUser,SalesYear,SalesMonth,Price,간편식사_도시락,간편식사_샌드위치,간편식사_주먹밥,과자류_껌/초,과자류_빵/디,과자류_스낵,식품_가공식,식품_식재료,식품_안주류,아이스크_아이스,음료_아이스,음료_유제품,음료_음료,즉석조리_베이커,즉석조리_즉석커,즉석조리_튀김류,Gender_남성,Gender_여성,AgeGroup_10대,AgeGroup_20대,AgeGroup_30대,AgeGroup_40대,AgeGroup_50대,AgeGroup_60대이상
0,U100001,2021,1,1000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0
1,U100001,2021,1,6500,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0
2,U100001,2021,1,750,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0
3,U100277,2022,10,5300,0,0,0,0,0,0,0,0,0,0,1,0,2,0,0,0,0,1,0,0,1,0,0,0
4,U100277,2022,10,1500,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0


## idUser가 SalesMonth에 구매한 중분류의 sum

In [43]:
df_raw.drop(df_raw.loc[:,'간편식사_도시락':'즉석조리_튀김류'], axis=1)

,idUser,SalesYear,SalesMonth,Price,Gender_남성,Gender_여성,AgeGroup_10대,AgeGroup_20대,AgeGroup_30대,AgeGroup_40대,AgeGroup_50대,AgeGroup_60대이상
0,U100001,2021,1,1000,1,0,0,0,0,1,0,0
1,U100001,2021,1,6500,1,0,0,0,0,1,0,0
2,U100001,2021,1,750,1,0,0,0,0,1,0,0
3,U100277,2022,10,5300,0,1,0,0,1,0,0,0
4,U100277,2022,10,1500,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
215613,U101098,2022,9,5200,1,0,0,0,1,0,0,0
215614,U101098,2022,9,3700,1,0,0,0,1,0,0,0
215615,U101098,2022,9,2900,1,0,0,0,1,0,0,0
215616,U101098,2022,9,10300,1,0,0,0,1,0,0,0


In [46]:
df_year_month

,idUser,SalesYear,SalesMonth,Price,간편식사_도시락,간편식사_샌드위치,간편식사_주먹밥,과자류_껌/초,과자류_빵/디,과자류_스낵,식품_가공식,식품_식재료,식품_안주류,아이스크_아이스,음료_아이스,음료_유제품,음료_음료,즉석조리_베이커,즉석조리_즉석커,즉석조리_튀김류,VisitCount
0,U100001,2021,1,8250,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,3
1,U100277,2022,4,47600,0,1,0,0,3,1,4,1,1,0,0,0,1,1,0,1,10
2,U100277,2022,5,138600,0,2,1,6,3,4,8,8,3,3,2,0,7,1,0,2,16
3,U100277,2022,6,127150,1,0,0,2,6,3,10,3,5,3,2,1,7,0,0,0,19
4,U100277,2022,7,248800,0,1,1,11,12,8,10,12,4,6,1,1,4,0,0,1,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18187,U101098,2022,8,89000,1,1,1,2,6,0,7,5,2,2,0,1,4,0,0,0,10
18188,U101098,2022,9,39050,0,0,1,2,2,4,3,3,0,1,1,0,0,0,0,0,8
18189,U101098,2022,10,102290,2,1,0,3,3,2,6,4,2,1,0,0,3,1,0,0,10
18190,U101098,2022,11,131280,1,1,0,1,3,1,8,4,3,3,0,0,3,1,0,1,13


In [53]:
df_year_month = df_raw.loc[:,:'즉석조리_튀김류']
df_year_month_user = df_raw.drop(df_raw.loc[:,'Price':'즉석조리_튀김류'], axis=1)

df_year_month_sum = df_year_month.groupby(["idUser",'SalesYear','SalesMonth']).agg('sum').reset_index()
# .loc[:, '간편식사_도시락':'즉석조리_튀김류']
# df_merged

## 방문 횟수
df_year_month_count = df_year_month.groupby(["idUser",'SalesYear','SalesMonth'])['Price'].agg('count').reset_index()


# pd.concat([df_year_month_sum, df_year_month_count], axis=1, keys=['idUser', 'SalesYear','SalesMonth'], join='inner')
df_year_month = pd.merge(df_year_month_sum, df_year_month_count, on=['idUser', 'SalesYear','SalesMonth'], how='left')
df_year_month.rename(columns={ 'Price_x' : 'Price',
                               'Price_y' : 'VisitCount'}, inplace=True)
df_year_month =  pd.merge(df_year_month, df_year_month_user, on=['idUser', 'SalesYear','SalesMonth'], how='left').drop_duplicates()
df_year_month.head()

,idUser,SalesYear,SalesMonth,Price,간편식사_도시락,간편식사_샌드위치,간편식사_주먹밥,과자류_껌/초,과자류_빵/디,과자류_스낵,식품_가공식,식품_식재료,식품_안주류,아이스크_아이스,음료_아이스,음료_유제품,음료_음료,즉석조리_베이커,즉석조리_즉석커,즉석조리_튀김류,VisitCount,Gender_남성,Gender_여성,AgeGroup_10대,AgeGroup_20대,AgeGroup_30대,AgeGroup_40대,AgeGroup_50대,AgeGroup_60대이상
0,U100001,2021,1,8250,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,3,1,0,0,0,0,1,0,0
3,U100277,2022,4,47600,0,1,0,0,3,1,4,1,1,0,0,0,1,1,0,1,10,0,1,0,0,1,0,0,0
13,U100277,2022,5,138600,0,2,1,6,3,4,8,8,3,3,2,0,7,1,0,2,16,0,1,0,0,1,0,0,0
29,U100277,2022,6,127150,1,0,0,2,6,3,10,3,5,3,2,1,7,0,0,0,19,0,1,0,0,1,0,0,0
48,U100277,2022,7,248800,0,1,1,11,12,8,10,12,4,6,1,1,4,0,0,1,24,0,1,0,0,1,0,0,0


In [54]:
df_year_month.shape

(18192, 29)

## SalesMonth의 월평균 온도/습도/강수량, 월 합계 강수량

In [55]:
df_weather_month = df_weather_raw.groupby(['dateObs_y','dateObs_m']).agg('mean')[['TempAvg', 'RainfallAmount','HumidAvg']].reset_index()
df_weather_month.columns = ['SalesYear', 'SalesMonth', 'TempAvg', 'RainfallAmount', 'HumidAvg']
df_weather_month = df_weather_month[ (df_weather_month['SalesYear']==2021)  | (df_weather_month['SalesYear']==2022) ]
df_weather_month

/tmp/ipykernel_5917/4219757885.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_weather_month = df_weather_raw.groupby(['dateObs_y','dateObs_m']).agg('mean')[['TempAvg', 'RainfallAmount','HumidAvg']].reset_index()


,SalesYear,SalesMonth,TempAvg,RainfallAmount,HumidAvg
12,2021,1,-2.370968,0.609677,57.451613
13,2021,2,2.732143,0.253571,55.439286
14,2021,3,9.016129,3.577419,62.845161
15,2021,4,14.196667,4.136667,54.210000
16,2021,5,17.067742,5.906452,68.148387
17,2021,6,22.816667,3.486667,72.496667
18,2021,7,28.119355,5.429032,70.909677
19,2021,8,25.858065,6.812903,74.225806
20,2021,9,22.596667,4.366667,70.883333
21,2021,10,15.574194,1.838710,69.722581


In [58]:
df_year_month = pd.merge(df_year_month, df_weather_month, on=['SalesYear','SalesMonth'], how='left')
df_year_month.to_csv("data/data_for_modeling_v2.csv",index=False)

In [45]:
# df_merged.to_csv("/home/piai/바탕화면/data_for_modeling_v1.csv")